# Reducing number of parameters (LeNet-5)
This notebook shows how to reduce the number of parameters of a network. It assumes 
that a trained LeNet-5 model already exists in the ```Models``` directory. You can use the 
notebook [Handwritten Digit recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb) to create and train a LeNet-5 model.

## Load the pretrained model

In [1]:
from fireball import Model

model = Model.makeFromFile("Models/LeNet5.fbm", gpus='0')   
model.initSession()
model.printLayersInfo()



Reading from "Models/LeNet5.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    156        
L2_CONV          14 14 6       KSP: 5 1 v               5 5 16        ReLU     MP(KSP):2 2 v    2,416      
L3_FC            5 5 16                                 120           ReLU                      48,120     
L4_FC            120                                    84            ReLU                      10,164     
L5_FC            84                                     10            None                      850        
OUT_CLASS        10            10 classes  

## Reducing number of parameters
Here we apply Low-Rank Decomposition on different layers of the model to reduce the number of parameters. We first create a list of layers we want to apply Low-Rank Decomposition, specify our tolerance (MSE), and pass this information to the [createLrModel](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.createLrModel) method. This creates the new fireball model saved to the file LeNet5R.fbm.

In [2]:
import time

layers = ['L2_CONV','L3_FC','L4_FC']
mse = 0.01
layerParams = [ (layer, mse) for layer in layers]

print('Now reducing number of network parameters ... ')
t0 = time.time()
model.createLrModel("Models/LeNet5R.fbm", layerParams)
print('Done. (%.2f Seconds)'%(time.time()-t0))

Now reducing number of network parameters ... 
  L2_CONV => LR(8), MSE=0.007843, Shape: (150, 16), Params: 2400->1328 (Reduction: 44.7%)
  L3_FC => LR(8), MSE=0.006996, Params: 48000->4160 (Reduction: 91.3%)
  L4_FC => LR(8), MSE=0.007918, Params: 10080->1632 (Reduction: 83.8%)
Total New Parameters: 8,346
Done. (0.17 Seconds)


## Evaluating the new model
Compare the new number of parameters with the original 61,706. Let's see the impact of this reduction to the performance of the model.

In [3]:
from fireball.datasets.mnist import MnistDSet

testDs = MnistDSet.makeDatasets('test', batchSize=128)

model = Model.makeFromFile("Models/LeNet5R.fbm", testDs=testDs, gpus='0')   
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/LeNet5R.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    156        
L2_CONV          14 14 6       KSP: 5 1 v, LR8          5 5 16        ReLU     MP(KSP):2 2 v    1,344      
L3_FC            5 5 16        LR8                      120           ReLU                      4,280      
L4_FC            120           LR8                      84            ReLU                      1,716      
L5_FC            84                                     10            None                      850        
OUT_CLASS        10            10 classes 

## Re-training after parameter reduction
Here we make a new model object from the file created above. We then call the [train](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.train) method of the model to start the re-training.

After re-training, we run the [evaluateDSet](https://interdigitalinc.github.io/Fireball/html/source/model.html#fireball.model.Model.evaluateDSet) function to see how re-training improves the performance
of the model.

The following code also shows how to save the model during the training. Here we save the model at the end of every epoch (```savePeriod=1```). Setting ```saveBest``` to ```True``` makes this function to keep a copy of the best model so far in a file with a different name.

In [4]:
import os
if os.path.exists( "Models/LeNet5RR.fbm" ):
    os.remove( "Models/LeNet5RR.fbm" )    # Delete the model if it already exists

trainDs, validDs, testDs = MnistDSet.makeDatasets('train,valid,test', batchSize=128)

model = Model.makeFromFile("Models/LeNet5R.fbm",
                           trainDs=trainDs, validationDs=validDs, 
                           optimizer='Adam',
                           numEpochs=5,
                           learningRate=(0.01,0.0001),
                           saveModelFileName="Models/LeNet5RR.fbm",  # Save the re-training ...
                           savePeriod=1, saveBest=False,             # ... every epoch
                           gpus='0')
model.printLayersInfo()
model.initSession()
model.train()
results = model.evaluateDSet(testDs)


Reading from "Models/LeNet5R.fbm" ... Done.
Creating the fireball model "LeNet-5" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 28x28x1      28 28 1       None                      0          
L1_CONV          28 28 1       KSP: 5 1 s               14 14 6       ReLU     MP(KSP):2 2 v    156        
L2_CONV          14 14 6       KSP: 5 1 v, LR8          5 5 16        ReLU     MP(KSP):2 2 v    1,344      
L3_FC            5 5 16        LR8                      120           ReLU                      4,280      
L4_FC            120           LR8                      84            ReLU                      1,716      
L5_FC            84                                     10            None                      850        
OUT_CLASS        10            10 classes 

## Also look at

[Pruning LeNet-5 Model](LeNet5-MNIST-Prune.ipynb)

[Quantizing LeNet-5 Model](LeNet5-MNIST-Quantize.ipynb)

[Exporting LeNet-5 Model to ONNX](LeNet5-MNIST-ONNX.ipynb)

[Exporting LeNet-5 Model to TensorFlow](LeNet5-MNIST-TF.ipynb)

[Hand-written Didgit Recognition as a Regression problem](Regression.ipynb)
________________

[Fireball Playgrounds](../Contents.ipynb)

[Handwritten Digit Recognition (LeNet-5/MNIST)](LeNet5-MNIST.ipynb)
